In [1]:
from numpy import *
import numpy
import scipy
import scipy.stats as stats
import matplotlib.pyplot as pyplot
import pandas
import datetime
import seaborn

# machine learning
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
import sklearn

import sys
sys.path.append("/Users/jcipar/Code/research-analysis/python-utils")
from rtbpythonutils import *
%matplotlib inline

In [ ]:
feature_columns = ['app_site_id', 'publisher_id', 'fiksu_app_id', 'creative_id', 'lam_status', 'ad_type', 'ad_size', 'ad_network',
                   'device_country', 'fiksu_device', 'connection_speed', 'monthly_imps', 'device_osv']
negative_ratio = 1.0

positives = redshift_compute('''
select case when conversions > 0 then 1 else 0 end as label,
  %s
from research_prod.bid_summary_4wk
where fiksu_app_id != '0'
  and used_target = 'cost_per_download'
  and conversions > 0
order by random()
limit 100000
'''%(', '.join(feature_columns)))

negatives = redshift_compute('''
select case when conversions > 0 then 1 else 0 end as label,
  %s
from research_prod.bid_summary_4wk
where fiksu_app_id != '0'
  and used_target = 'cost_per_download'
  and conversions = 0
  and impressions > 0
order by random()
limit %s
'''%(', '.join(feature_columns), int(len(positives)*negative_ratio)))
all_data = pandas.concat([positives, negatives])

def random_split(X,Y, frac=0.8):
    msk = numpy.random.rand(len(X)) < frac
    return X[msk], Y[msk], X[~msk], Y[~msk]

In [ ]:
print(all_data.columns.values)

all_data.describe()

In [ ]:
columns = ['app_aid']
X = pandas.get_dummies(all_data[columns])
Y = all_data['label']

X_tmp, Y_tmp, X_validation, Y_validation = random_split(X,Y)
X_train, Y_train, X_test, Y_test = random_split(X_tmp, Y_tmp)


In [ ]:
method = "Logistic Regression"


model = LogisticRegression()
model.fit(X_train, Y_train)

train_acc = round(model.score(X_train, Y_train) * 100, 2)
test_acc = round(model.score(X_test, Y_test) * 100, 2)

train_loss = sklearn.metrics.log_loss(Y_train, model.predict_proba(X_train)[:,1])
test_loss = sklearn.metrics.log_loss(Y_test, model.predict_proba(X_test)[:,1])

mean_loss = sklearn.metrics.log_loss(Y_test, ones(len(Y_test)) * mean(Y_train))


print(method)
print("Training Accuracy", train_acc)
print("Test Accuracy", test_acc)
print("Training Loss", train_loss)
print("Test Loss", test_loss)
print("Mean Loss", mean_loss)

In [ ]:

columns = ['app_site_id', 'fiksu_app_id', 'ad_type', 'ad_size', 'lam_status']
X = pandas.get_dummies(all_data[columns])
Y = all_data['label']

X_train, Y_train, X_test, Y_test = random_split(X, Y)

In [ ]:
method = "Logistic Regression"


model = LogisticRegression()
model.fit(X_train, Y_train)

train_acc = round(model.score(X_train, Y_train) * 100, 2)
test_acc = round(model.score(X_test, Y_test) * 100, 2)

train_loss = sklearn.metrics.log_loss(Y_train, model.predict_proba(X_train)[:,1])
test_loss = sklearn.metrics.log_loss(Y_test, model.predict_proba(X_test)[:,1])

mean_loss = sklearn.metrics.log_loss(Y_test, ones(len(Y_test)) * mean(Y_train))


print(method)
print("Training Accuracy", train_acc)
print("Test Accuracy", test_acc)
print("Training Loss", train_loss)
print("Test Loss", test_loss)
print("Mean Loss", mean_loss)

In [ ]:
method = "Decision Tree"

model = DecisionTreeClassifier()
model.fit(X_train, Y_train)

train_acc = round(model.score(X_train, Y_train) * 100, 2)
test_acc = round(model.score(X_test, Y_test) * 100, 2)

train_loss = sklearn.metrics.log_loss(Y_train, model.predict_proba(X_train)[:,1])
test_loss = sklearn.metrics.log_loss(Y_test, model.predict_proba(X_test)[:,1])

mean_loss = sklearn.metrics.log_loss(Y_test, ones(len(Y_test)) * mean(Y_train))


print(method)
print("Training Accuracy", train_acc)
print("Test Accuracy", test_acc)
print("Training Loss", train_loss)
print("Test Loss", test_loss)
print("Mean Loss", mean_loss)

In [ ]:
method="Random Forest"

model = RandomForestClassifier(n_estimators=100)
model.fit(X_train, Y_train)

train_acc = round(model.score(X_train, Y_train) * 100, 2)
test_acc = round(model.score(X_test, Y_test) * 100, 2)

train_loss = sklearn.metrics.log_loss(Y_train, model.predict_proba(X_train)[:,1])
test_loss = sklearn.metrics.log_loss(Y_test, model.predict_proba(X_test)[:,1])

mean_loss = sklearn.metrics.log_loss(Y_test, ones(len(Y_test)) * mean(Y_train))


print(method)
print("Training Accuracy", train_acc)
print("Test Accuracy", test_acc)
print("Training Loss", train_loss)
print("Test Loss", test_loss)
print("Mean Loss", mean_loss)